In [1]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate
from aocutils.maze import bfs, dijkstra, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

In [2]:

def plot(grid):
    global found
    if found < 10:
        if isinstance(grid, list):
            grid = np.array(grid)
        fig = plt.figure(figsize=(190,10))    
        # fig.set_size_inches(10, 15)
        fig.savefig('image.png')
        plt.figure(figsize=(10,4))
        # plt.imsave('test'+str(found)+'.png', grid)
        plt.imshow(grid)
    else:
        sys.exit()

In [3]:
  
from itertools import cycle
def p(taken):
    global curhigh
    grid = [[0 for _ in range(7)] for _ in range(62)]

    for r,c in taken:
        grid[curhigh-r][c] = 1

    plot(grid)

In [6]:
found = 0
rightwall = 7
leftwall = -1
curhigh = 0
taken = set()
rock1 = ((0,0),(0,1),(0,2),(0,3))
rock2 = ((1,0),(1,1),(1,2),(0,1),(2,1))
rock3 = ((0,0),(0,1),(0,2),(1,2),(2,2))
rock4 = ((0,0),(1,0),(2,0),(3,0))
rock5 = ((0,0),(0,1),(1,0),(1,1))
rocks = cycle([rock1,rock2,rock3,rock4,rock5])
warmupcycles = 0

w = open("t0.txt").read()
wind = cycle(w)
tops = {}
minn = 0
MOD = lcm((len(w),5))
import sys
def possible(rock):
    for r,c in rock:
        if r <= 0:
            return False
        if c < 0 or c > 6:
            return False
        if (r,c) in taken:
            return False
    return True

def step(idx):
    global curhigh, taken,minn, prevtop, found, windidx
    rock = next(rocks)
    rock = [(r+curhigh+4, c +2) for r,c in rock]
    # print('newrock',rock)
    steppies = 0
    way = ''
    
    while True:
        # print('cur', rock)
        win = next(wind)
        windidx = (windidx+ 1) % len(w)
        steppies += 1
        if win == '>':
            newrock = [(r, c+1) for r,c in rock]
            if possible(newrock):
                way += '0'
                rock = newrock
            else:
                way += '1'
        else:
            newrock = [(r, c-1) for r,c in rock]
            if possible(newrock):
                way += '2'
                rock = newrock
            else:
                way += '3'
        # fall

        newrock = [(r-1, c) for r,c in rock]
        if possible(newrock):
            way += '4'
            rock = newrock
        else:
            way += '5'
            break
    # print('stop', rock)
    newhigh = curhigh
    for node in rock:
        taken.add(node)
    newhigh = max(curhigh, max(r for r,c in rock))
    curhigh = newhigh
    newtop = []
    for c in range(7):
        newtop.append(max([r for r,c2 in taken if c==c2], default=0)-curhigh)
        # mintop.append(min([r for r,c2 in taken if c==c2], default=0)-curhigh)
    minn = min(minn, min(newtop))
    taken = {(r,c) for r,c in taken if r-curhigh >= -60}
    # toadd = (prevtop, tuple(newtop), frozenset({(r-curhigh, c) for r,c in taken if r-curhigh >= minn}), windidx, rockidx)
    toadd = (prevtop, tuple(newtop), windidx, rockidx, steppies, way)
    # print('toadd', toadd)
    if idx == warmupcycles:
        print('warmup', curhigh)
    if toadd in tops:
        prevhigh, previdx = tops[toadd]
        deltahigh = curhigh - prevhigh
        deltaidx = idx - previdx
        print('len', len(taken))
        print('toadd', toadd, curhigh, idx)
        print('curidx', idx, 'curhigh', curhigh)
        print('this was previous startidx', tops[toadd][1], 'curhigh', tops[toadd][0])
        print('increase', deltahigh, 'mod', deltaidx)
        found += 1
        # p(taken)
        if found == 50:
            sys.exit()
        

        
        tops[toadd] = curhigh, idx
    elif idx >= warmupcycles:
        
        tops[toadd] = curhigh, idx
    # if idx in [1056,1057,1058,152421,152422,152423]:
    #     print('FOUND', toadd, curhigh, idx)
    #     p(taken)
    prevtop = tuple(newtop)
  
prevtop = None
windidx = 0
rockidx = 0
for idx in range(1,211111111023):
    if idx % 10000 == 0:
        print(idx)
    
    step(idx)
    # windidx =  (windidx+ 1) % len(w)
    rockidx = (rockidx + 1) % 5
print(curhigh)
p(taken)

display = 2

len 172
toadd ((-2, -2, 0, -2, -3, -5, -7), (0, -4, -2, -4, -5, -7, -9), 34, 3, 6, '242434041425') 104 64
curidx 64 curhigh 104
this was previous startidx 29 curhigh 51
increase 53 mod 35
len 176
toadd ((0, -4, -2, -4, -5, -7, -9), (0, -4, -2, -4, -5, -5, -5), 5, 4, 11, '0404242404040414143435') 104 65
curidx 65 curhigh 104
this was previous startidx 30 curhigh 51
increase 53 mod 35
len 180
toadd ((0, -4, -2, -4, -5, -5, -5), (0, -1, -1, -1, -1, -5, -5), 11, 0, 6, '042404142425') 104 66
curidx 66 curhigh 104
this was previous startidx 31 curhigh 51
increase 53 mod 35
len 178
toadd ((0, -1, -1, -1, -1, -5, -5), (-2, -3, -3, -1, 0, -1, -7), 16, 1, 5, '2404042405') 106 67
curidx 67 curhigh 106
this was previous startidx 32 curhigh 53
increase 53 mod 35
len 172
toadd ((-2, -3, -3, -1, 0, -1, -7), (-5, -6, -2, -2, 0, -4, -10), 20, 2, 4, '04042425') 109 68
curidx 68 curhigh 109
this was previous startidx 33 curhigh 56
increase 53 mod 35
len 164
toadd ((-5, -6, -2, -2, 0, -4, -10), (-9, -10, 

SystemExit: 

In [ ]:
this was previous startidx 29 curhigh 51
increase 53 mod 35

In [8]:
want = 1000000000000


#test
increase = 53
mod = 35
warmupidx = 29 # is not used atm!
warmuphigh = 51
#true
# warmup = 0
# increase = 2548  
# mod = 1690
# warmupidx = 109
# warmuphigh = 169

want -= warmupcycles
amount, restcycles = divmod(want, mod)
res = warmuphigh + amount * increase
toadd = False
for high, idx in tops.values():
    if idx == restcycles+warmupcycles:
        toadd = high-warmuphigh
if not toadd:
    print('error')
print(res + toadd == 1514285714288)
res + toadd

True


1514285714288

In [ ]:
curhigh and idx 245101 162562
starttop and startidx (16919, 11197)
increase 228182 mod 151365

In [ ]:
warmupidx

15148

In [ ]:
increase = 424
mod = 280
warmupidx = 27
warmuphigh = 49

(1583333333229, 63)

In [ ]:
# 1583333333330 too high
1507759389583
1507759390523 too high
1507495129520 not right
1507692307690




True

In [ ]:
3148